In [1]:
import os
import argparse
import json
import shutil

import numpy as np
import torch
import skvideo.io

from processor.io import IO
import tools
import tools.utils as utils

from IPython.display import clear_output

In [2]:
p = IO()
# clear_output()

Init Start
self.get_parser finish
yaml load finish
Load config Finish
Load arg finish
Kog gak sampai sini ............................
Mid of init enviroment


/home/simslab-cs/Documents/lab-project/repository/st-gcn-debug1/processor/io.py:46: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  default_arg = yaml.load(f)


Load init_environment finish
Before load model
Before import_class
mod_str =  net.st_gcn
Model
Before import **model_args
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
Model(
  (data_bn): BatchNorm1d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (st_gcn_networks): ModuleList(
    (0): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(3, 192, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(9, 1), stride=(1, 1), padding=(4, 0))
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Dropout(p=0, inplace)
      )
      (relu): ReLU(inplace)
    )
    (1): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(64, 192, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps

In [3]:
p.model.eval()

Model(
  (data_bn): BatchNorm1d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (st_gcn_networks): ModuleList(
    (0): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(3, 192, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(9, 1), stride=(1, 1), padding=(4, 0))
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Dropout(p=0, inplace)
      )
      (relu): ReLU(inplace)
    )
    (1): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(64, 192, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(9, 1), stride=(1, 1), padding=(4

In [4]:
video = utils.video.get_video_frames('/media/simslab-cs/A/videoset/cgutest.mp4')
# video = utils.video.get_video_frames('/media/simslab-cs/A/videoset/F6_cut/F6_082652_082730.avi')
height, width, _ = video[0].shape

In [8]:
# video_info = utils.openpose.json_pack(output_snippets_dir, video_name, width, height)
file_json = 'cgu_test2.json'
with open(file_json, encoding='utf-8') as data_file:
    data = json.loads(data_file.read())
sequence_info = []
weight = data.get('## description').get('image_width')
height = data.get('## description').get('image_height')
for frame_number in data.keys():
    try:
        frame_number = int(frame_number)
    except Exception as e:
        continue
    #Get highest possibilites human confiden score --> usualy at detection no 0
    id_name = list(data.get(str(frame_number)).keys())[2]
    
    frame_id = frame_number
    frame_data = {'frame_index': frame_id}
    skeletons = []
    
    score, coordinates = [], []
    skeleton = {}
    
    
    for part in data.get(str(frame_number)).get(id_name):
        coordinates += [int(part.get('position')[0]*weight+0.5),int(part.get('position')[1]*height+0.5)]
        score += [part.get('score')]
    skeleton['pose'] = coordinates
    skeleton['score'] = score
    skeletons +=[skeleton]
    frame_data['skeleton'] = skeletons
    sequence_info += [frame_data]

video_info = dict()
video_info['data'] = sequence_info
video_info['label'] = 'unknowns'
video_info['label_index'] = -1

In [9]:
pose, _ = utils.video.video_info_parsing(video_info)

In [10]:
p.model.graph.get_adjacency('uniform')

In [11]:
p.arg

Namespace(config='config/st_gcn/kinetics-skeleton/demo.yaml', device=[0], ignore_weights=[], model='net.st_gcn.Model', model_args={'in_channels': 3, 'num_class': 400, 'edge_importance_weighting': True, 'graph_args': {'layout': 'openpose', 'strategy': 'spatial'}}, print_log=True, save_log=True, use_gpu=True, weights='./models/st_gcn.kinetics.pt', work_dir='./work_dir/tmp')

In [12]:
data = torch.from_numpy(pose)
data = data.unsqueeze(0)
data = data.float().to(p.dev).detach()

In [13]:
output, feature = p.model.extract_feature(data)

In [14]:
p.model.graph.A

array([[[0.25      , 0.25      , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.33333333,
         0.33333333, 0.        , 0.        ],
        [0.25      , 0.25      , 0.25      , 0.        , 0.        ,
         0.25      , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.25      , 0.25      , 0.33333333, 0.        ,
         0.        , 0.        , 0.        , 0.33333333, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.25      , 0.33333333, 0.5       ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [15]:
output = output[0]
feature = feature[0]
intensity = (feature*feature).sum(dim=0)**0.5
intensity = intensity.cpu().detach().numpy()
label = output.sum(dim=3).sum(dim=2).sum(dim=1).argmax(dim=0)

In [16]:
label_name_path = './resource/kinetics_skeleton/label_name.txt'
with open(label_name_path) as f:
    label_name = f.readlines()
    label_name = [line.rstrip() for line in label_name]
print('Prediction result: {}'.format(label_name[label]))
print('Done.')

Prediction result: archery
Done.


In [17]:
label = output.sum(dim=3).sum(dim=2).sum(dim=1)
lavel = label.cpu().detach().numpy()
idx = lavel.argsort()[-5:][::-1]

In [18]:
#Other detection 
for i in range(5):
    print('Prediction result: {}'.format(label_name[idx[i]]))

Prediction result: archery
Prediction result: feeding birds
Prediction result: catching or throwing baseball
Prediction result: throwing ball
Prediction result: headbutting


In [19]:
#Other detection 
for i in range(5):
    print('Prediction result: {}'.format(label_name[idx[i]]))

Prediction result: archery
Prediction result: feeding birds
Prediction result: catching or throwing baseball
Prediction result: throwing ball
Prediction result: headbutting
